In [31]:
from sklearn.feature_selection import SelectKBest
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import RobustScaler
from sklearn.feature_selection import mutual_info_classif
import numpy as np
import pandas as pd

In [29]:
with open("./Dataset/x_test.txt") as file:
    X_test = [[float(digit) for digit in line.split()] for line in file]


with open("./Dataset/x_train.txt") as file:
    X_train = [[float(digit) for digit in line.split()] for line in file]


with open("./Dataset/y_train.txt") as file:
    y_train = [[float(digit) for digit in line.split()] for line in file]

In [30]:
y_train_ravel = np.ravel(y_train, order="C")
y_train_ravel = y_train_ravel.astype(int)

In [32]:
class FinalModel:
    """Model with selector, scaler and a classifier that have been chosen
    as a result of all the experiments
    """

    def __init__(self):
        self.selector = SelectKBest(k=2, score_func=mutual_info_classif)

        self.scaler = RobustScaler()

        self.classifier = MLPClassifier(
            activation="relu",
            beta_1=0.95,
            beta_2=0.999,
            solver="adam",
            alpha=0.3,
            learning_rate="adaptive",
            learning_rate_init=0.008,
            hidden_layer_sizes=(13,),
            max_iter=1600,
            random_state=42,
        )

    def fit(self, X_t, y_t):
        X_new = self.selector.fit_transform(X_t, y_t)
        X_new = self.scaler.fit_transform(X_new, y_t)
        self.classifier.fit(X_new, y_t)

    def predict(self, X_te, limitCount):
        X_new = self.selector.transform(X_te)
        X_new = self.scaler.fit_transform(X_new)
        y_pred = self.classifier.predict_proba(X_new)
        df = pd.DataFrame(y_pred[:, 1], columns=["result"])
        sortedDf = df.sort_values(by="result", ascending=False).head(limitCount)

        y_predicted = np.zeros(len(X_new))

        y_predicted[sortedDf.index] = 1

        selectedFeaturesIndexes = self.selector.get_support(indices=True)

        return selectedFeaturesIndexes, y_predicted

In [33]:
finalModel = FinalModel()

finalModel.fit(X_train, y_train_ravel)

featuresIndexes0Based, predictions = finalModel.predict(X_test, 1000)

In [34]:
predictionsIndexes0Based = np.where(predictions == 1)[0]

predictionsIndexes1Based = predictionsIndexes0Based + 1

featuresIndexes1Based = featuresIndexes0Based + 1

In [44]:
np.savetxt(
    "featuresIndexes1Based.txt",
    np.array(featuresIndexes1Based, dtype=int),
    newline="\n",
    fmt="%d",
)
np.savetxt(
    "predictionsIndexes1Based.txt",
    np.array(predictionsIndexes1Based, dtype=int),
    newline="\n",
    fmt="%d",
)

In [35]:
featuresIndexes1Based

array([102, 137], dtype=int64)

In [36]:
predictionsIndexes1Based

array([   3,    4,    5,    7,   12,   14,   21,   24,   25,   31,   34,
         39,   43,   44,   55,   64,   83,   85,   88,   89,   99,  103,
        116,  130,  131,  142,  145,  150,  151,  162,  166,  186,  189,
        193,  198,  206,  207,  216,  217,  222,  223,  224,  227,  230,
        237,  241,  245,  261,  271,  273,  280,  281,  285,  287,  288,
        290,  300,  302,  315,  325,  326,  336,  339,  340,  343,  347,
        356,  357,  358,  363,  364,  373,  374,  377,  391,  393,  399,
        412,  415,  419,  424,  427,  429,  435,  440,  461,  469,  472,
        475,  477,  482,  488,  491,  492,  497,  500,  507,  514,  524,
        527,  531,  538,  551,  558,  560,  576,  578,  580,  590,  612,
        616,  617,  619,  644,  649,  650,  656,  660,  662,  678,  679,
        684,  685,  690,  692,  703,  704,  705,  707,  724,  725,  727,
        732,  733,  740,  742,  749,  752,  764,  765,  769,  773,  776,
        777,  783,  788,  792,  794,  795,  797,  8